<a href="https://colab.research.google.com/github/EduardoMetzger/Colab/blob/main/BRZ001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
import glob
import os
import pytz
from datetime import datetime

# Migra os arquivos salvos pelo crawler para a BRZ001 (tabela que contém os dados originais e completos de captura por dia)

def create_db_connection(db_file):
    """ Cria uma conexão com o banco de dados SQLite especificado. """
    try:
        conn = sqlite3.connect(db_file)
        print("Conexão estabelecida com SQLite")
        return conn
    except Exception as e:
        print(e)
        return None

def create_table(conn, table_name):
    """ Cria uma tabela no banco de dados especificado pela string SQL. """
    try:
        c = conn.cursor()
        c.execute(f'''
            CREATE TABLE IF NOT EXISTS {table_name} (
                URLPRO TEXT,
                IMGPRO TEXT,
                PAGPRO TEXT,
                NOMPRO TEXT,
                VLRDDE TEXT,
                VLRPOR TEXT,
                QTVEND TEXT,
                AVAMED TEXT,
                QTAVAL TEXT,
                NOMVEN TEXT,
                TAMVEN TEXT,
                CARPRO TEXT,
                DATCAP TEXT,
                HORCAP TEXT,
                INFORI TEXT,
                DATGER TEXT,
                HORGER TEXT,
                PRIMARY KEY (URLPRO, DATCAP, HORCAP)
            );
        ''')
        conn.commit()
    except Exception as e:
        print(e)

def process_files_to_db(folder_path, db_file, table_name, output_excel_path, process_date=None):
    all_data = pd.DataFrame()  # DataFrame para acumular todos os dados
    conn = create_db_connection(db_file)

    # Se uma data de processamento não for fornecida, use a data atual
    if process_date is None:
        process_date = datetime.now().strftime('%Y%m%d')

    folder_path = os.path.join(folder_path, process_date)  # Define o caminho para a pasta do dia especificado
    pattern = os.path.join(folder_path, '*.xlsx')

    # Configuração do fuso horário de São Paulo
    timezone_sao_paulo = pytz.timezone('America/Sao_Paulo')

    # Carrega as chaves existentes da tabela para evitar inserir duplicatas
    existing_keys = pd.read_sql_query(f"SELECT URLPRO, DATCAP, HORCAP FROM {table_name}", conn)

    for filename in glob.glob(pattern):
        try:
            df = pd.read_excel(filename, engine='openpyxl')

            # Adicionando o caminho de informação
            df['INFORI'] = filename

            # Renomeando colunas
            df.rename(columns={
                'Href': 'URLPRO',
                'Src': 'IMGPRO',
                'Origem': 'PAGPRO',
                'Nome do Produto': 'NOMPRO',
                'Preço De': 'VLRDDE',
                'Preço Por': 'VLRPOR',
                'Quantidade Vendida': 'QTVEND',
                'Avaliação Final': 'AVAMED',
                'Quantidade de Avaliações': 'QTAVAL',
                'Vendido Por': 'NOMVEN',
                'Tamanho do Vendedor': 'TAMVEN',
                'Características do Produto': 'CARPRO',
                'Data e Hora da Captura': 'DATHOR'
            }, inplace=True)

            # Tratamento dos campos de data e hora
            df['DATHOR'] = pd.to_datetime(df['DATHOR'])
            df['DATCAP'] = df['DATHOR'].dt.strftime('%d/%m/%Y')
            df['HORCAP'] = df['DATHOR'].dt.strftime('%H:%M:%S')
            df.drop('DATHOR', axis=1, inplace=True)

            # Removendo caracteres especiais do campo QTAVAL
            df['QTAVAL'] = df['QTAVAL'].astype(str).str.replace('[()]', '', regex=True)

            # Convertendo todos os valores para texto
            for col in df.columns.difference(['DATCAP', 'HORCAP']):
                df[col] = df[col].astype(str)

            # Adicionando os campos de data e hora de geração com fuso horário de São Paulo
            now_sao_paulo = datetime.now(timezone_sao_paulo)
            df['DATGER'] = now_sao_paulo.strftime('%d/%m/%Y')
            df['HORGER'] = now_sao_paulo.strftime('%H:%M:%S')

            # Filtrar duplicatas baseando-se nas chaves
            keys = df[['URLPRO', 'DATCAP', 'HORCAP']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
            df = df[~keys.isin(existing_keys.set_index(['URLPRO', 'DATCAP', 'HORCAP']).index)]

            if not df.empty:
                df.to_sql(table_name, conn, if_exists='append', index=False)
                # Acumula os dados
                all_data = pd.concat([all_data, df], ignore_index=True)
        except Exception as e:
            print(f"Erro ao processar o arquivo {filename}: {e}")

    # Fecha a conexão com o banco de dados
    conn.close()

    # Ordena os dados combinados
    all_data.sort_values(by=['DATCAP', 'HORCAP'], ascending=[False, False], inplace=True)

    # Salva os dados acumulados em um arquivo Excel
    all_data.to_excel(output_excel_path, index=False)
    print(f"Todos os dados foram salvos em {output_excel_path}")

# Caminho para a pasta onde estão os arquivos, banco de dados e o arquivo Excel de saída
data_folder = '/content/drive/Othercomputers/Casa/hipometrics/scraping/_mercadoLivre/2.details'
db_file = '/content/drive/Othercomputers/Casa/hipometrics/database/bronze/BRZ001.db'
table_name = 'BRZ001'
output_excel_path = '/content/drive/Othercomputers/Casa/hipometrics/database/bronze/BRZ001_combined.xlsx'

# Se você quiser processar uma data específica, defina-a aqui no formato 'AAAAMMDD'.
# Se não, deixe process_date como None para usar a data atual.
process_date = None  # ou '20240414' para uma data específica

# Executar o script
conn = create_db_connection(db_file)
if conn is not None:
    create_table(conn, table_name)
    process_files_to_db(data_folder, db_file, table_name, output_excel_path, process_date)
    conn.close()


Conexão estabelecida com SQLite
Conexão estabelecida com SQLite
